In [1]:
import sqlite3
import numpy as np
import pandas as pd

In [2]:
db_con = sqlite3.connect('data/database.sqlite')
cursor = db_con.cursor()
cursor.execute('SELECT name FROM sqlite_master WHERE type="table"')
print(cursor.fetchall())

[('Reviews',)]


In [3]:
pos_neg_reviews = pd.read_sql_query('SELECT * FROM Reviews WHERE Score !=3 LIMIT 10000', db_con)

In [4]:
def assignPositive(x):
    if(x>3):
        return 1
    return 0

In [5]:
colScore = pos_neg_reviews['Score']
posNeg = colScore.map(assignPositive)
pos_neg_reviews['Score'] = posNeg
pos_neg_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
Id                        10000 non-null int64
ProductId                 10000 non-null object
UserId                    10000 non-null object
ProfileName               10000 non-null object
HelpfulnessNumerator      10000 non-null int64
HelpfulnessDenominator    10000 non-null int64
Score                     10000 non-null int64
Time                      10000 non-null int64
Summary                   10000 non-null object
Text                      10000 non-null object
dtypes: int64(5), object(5)
memory usage: 781.4+ KB


In [6]:
def dataCleaning(df):
    
    #Removing duplicates
    df.drop_duplicates(subset={'UserId','ProfileName','Time','Text'}, keep='first', inplace=True)
    
    #HelpfulnessDenominator is the sum of helpful(HelpfulnessNumerator) and no opinion
    df = df[df.HelpfulnessNumerator <= df.HelpfulnessDenominator]
    return df

In [7]:
readyData = dataCleaning(pos_neg_reviews)
readyData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9564 entries, 0 to 9999
Data columns (total 10 columns):
Id                        9564 non-null int64
ProductId                 9564 non-null object
UserId                    9564 non-null object
ProfileName               9564 non-null object
HelpfulnessNumerator      9564 non-null int64
HelpfulnessDenominator    9564 non-null int64
Score                     9564 non-null int64
Time                      9564 non-null int64
Summary                   9564 non-null object
Text                      9564 non-null object
dtypes: int64(5), object(5)
memory usage: 821.9+ KB


In [8]:
#https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python/47091490#47091490
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

def expandContractions(text):
    words = text.split(' ')
    expandedText = ''
    for word in words:
        if(word in contractions):
            expandedText += contractions[word].split('/')[0] #Always considering 1st form of verb for list
        else:
            expandedText += word
        expandedText += ' '
    return expandedText

In [9]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

#>>> import nltk
#>>> nltk.download()
#https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
stopWords = set(stopwords.words('english'))

#To save semantics
wordsRequiredOfStopWords = ['not', 'nor', 'neither' , 'no']

stopWords = stopWords.difference(wordsRequiredOfStopWords)

#print('I' in stopWords)

def getNonStopWordsLowerCase(text):
    textTokens = word_tokenize(text)
    retText = ''
    for w in textTokens:
        w = w.lower()
        if(w not in stopWords and len(w) > 1) :
            retText += w + ' '
            
    return retText
    

In [10]:
#https://lxml.de/
#https://www.crummy.com/software/BeautifulSoup/bs4/doc/#
#https://pypi.org/project/tqdm/2.2.3/

from bs4 import BeautifulSoup
import re
from tqdm import tqdm #show a progress meter - just wrap any iterable


def textCleaning(text):
    
    #Remove Hyperlinks
    # Regex: r'\S+', # sequence of non-whitespace
    text = re.sub(r'http\S+', '', text)
    
    #Remove all tags
    text = BeautifulSoup(text, 'html.parser').get_text()
    
    #Expand texts
    text = expandContractions(text)
    
    #Remove all words containing numbers
    #https://stackoverflow.com/questions/18082130/python-regex-to-remove-all-words-which-contains-number
    text = ' '.join(s for s in text.split() if not any(c.isdigit() for c in s))
    
    #Remove special characters
    text = re.sub('[^A-Za-z]+', ' ', text)
    
    #Get non stop words in lower case
    text = getNonStopWordsLowerCase(text).strip()
    
    return text

In [11]:
readyText = []
#text_full_sentence = ''

for text in tqdm(readyData['Text'].values):
    doneText = textCleaning(text)
    if(len(doneText) < 2):
        continue
    readyText.append(doneText)
    #text_full_sentence += doneText

100%|████████████████████████████████████████████████████████████████████████████| 9564/9564 [00:04<00:00, 1955.81it/s]


In [12]:
readySummary = []
for text in tqdm(readyData['Summary'].values):
    readySummary.append(textCleaning(text))

 80%|█████████████████████████████████████████████████████████████               | 7690/9564 [00:01<00:00, 5235.39it/s]C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:302: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
100%|████████████████████████████████████████████████████████████████████████████| 9564/9564 [00:01<00:00, 5374.69it/s]


In [13]:
#For Bag of Words

#http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
from sklearn.feature_extraction.text import CountVectorizer

textCountVect = CountVectorizer()
textCountVect.fit(readyText)
readyTextCountVect = textCountVect.transform(readyText)

In [14]:
#For bigrams | ngrams
textCountVect = CountVectorizer(ngram_range=(1,2), min_df=10, max_features=5000)
textCountVectBigram = textCountVect.fit_transform(readyText)


In [15]:
#For TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

model_tf_idf_textCountVect = TfidfVectorizer()#ngram_range=(1,2), min_df=10)
tf_idf_textCountVect = model_tf_idf_textCountVect.fit_transform(readyText)

In [16]:
#For Word2Vec
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

all_text_words = []
for text in readyText:
    all_text_words.append(text.split())
    
    #min_count: Minimum frequency of word to be considered
    #size: Dimensionality of feature vectors
    #workers: number of worker threads to train the model
w2vModel = Word2Vec(all_text_words,min_count=5,size=50,workers=4)

In [17]:
w2v_words = list(w2vModel.wv.vocab)
print("Total words in W2V Model:", len(w2v_words))

Total words in W2V Model: 5665


In [18]:
#Text to vector conversion: Average W2V
avg_w2v_text = []
for sentence in tqdm(all_text_words):
    w2v_vec = np.zeros(50) #size=50
    count = 0
    for word in sentence:
        if word in w2v_words:
            w2v_vec += w2vModel.wv[word]
            count += 1
    if(count != 0):
        w2v_vec /= count
    avg_w2v_text.append(w2v_vec)

100%|████████████████████████████████████████████████████████████████████████████| 9564/9564 [00:07<00:00, 1346.94it/s]


In [19]:
#Text to vector conversion: Weighted W2V with TF-IDF

word_idf_map = dict(zip(model_tf_idf_textCountVect.get_feature_names(), 
                       list(model_tf_idf_textCountVect.idf_)))
weightedW2V_text = []
for sentence in tqdm(all_text_words):
    w2v_vec = np.zeros(50) #size=50
    weighted_sum = 0
    for word in sentence:
        if word in w2v_words:
            tf_idf = word_idf_map[word] * (sentence.count(word)/len(sentence))
            w2v_vec += (w2vModel.wv[word] * tf_idf)
            weighted_sum += tf_idf
    if(weighted_sum != 0):
        w2v_vec /= weighted_sum
    weightedW2V_text.append(w2v_vec)

100%|████████████████████████████████████████████████████████████████████████████| 9564/9564 [00:08<00:00, 1109.15it/s]


In [20]:
weightedW2V_text

[array([-0.05384966,  0.695532  ,  0.02790565, -0.36573006, -0.38181838,
        -0.85077972, -0.44117292, -0.32753557, -0.07684316, -0.12870377,
         0.43608097, -0.16464817,  0.18870123,  0.68745294,  0.1607237 ,
        -0.26047092, -0.20947286,  0.08391118,  0.10475267, -0.22877957,
        -0.45734298, -0.19689699, -0.69047569, -0.63442207, -0.4601728 ,
        -0.44743115,  0.11391757, -0.86423337,  0.17305131, -0.18184781,
         0.48335851, -0.20829651, -0.55704945, -0.24828042, -0.18450411,
        -0.40831106,  0.12221418,  0.00664643,  0.03482452,  0.45635594,
         0.52956405, -0.25420511, -0.35133101, -0.18947519, -0.22752373,
        -0.20162163,  0.22122897,  0.42294368,  0.58667248,  0.78240907]),
 array([-0.08728471,  0.45865086,  0.12961382, -0.06331137, -0.04692313,
        -0.41993534, -0.20981958, -0.29971273,  0.04495622, -0.08359159,
         0.11755076, -0.05964334,  0.05553408,  0.40106409,  0.0264188 ,
        -0.11096328, -0.12790993,  0.04525231,  0